In [1]:
import sqlite3
import pickle
import pandas as pd
import numpy as np
import spacy
import stanza
import re
import string
import argparse

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df[df['text'].isna()]

,Unnamed: 0,tweet_id,text
671,1409,11230271,NaN
1132,2528,47140692,NaN
1628,3603,66010962,NaN
2298,4768,88051252,NaN
2660,5325,97514182,NaN
3175,6156,109542962,NaN
3285,6369,112971462,NaN
3344,6474,114817972,NaN
3737,7006,121659062,NaN
4075,7504,126644672,NaN


In [6]:
df[df['tweet_id']==11230271]['text'].isna().values[0]

True

In [2]:
stanza.download('it')

2023-11-30 15:06:02 INFO: Downloading default packages for language: it (Italian) ...
2023-11-30 15:06:03 INFO: File exists: /g100/home/userexternal/ddurmush/stanza_resources/it/default.zip
2023-11-30 15:06:08 INFO: Finished downloading models and saved to /g100/home/userexternal/ddurmush/stanza_resources.


In [5]:
### for future how to display whole tweets 
pd.set_option('display.max_colwidth', None)

In [3]:
### just a random dataset to analyze
with open('/g100_work/IscrC_mental/data/user_classification/user_age_gender_location_test_set.pkl', 'rb') as file:
    data_test = pickle.load(file)
tweets = list(data_test['tweet'])
tweets = [str(tw) for tw in tweets]

In [15]:
it_nlp_spacy = spacy.load('it_core_news_lg')

components_to_remove = ['tagger', 'parser', 'attribute_ruler', 'ner']
for component in components_to_remove:
    it_nlp_spacy.remove_pipe(component)
    
    
it_nlp_stanza = stanza.Pipeline('it', processors='tokenize,mwt,pos,lemma', verbose=False, use_gpu=False)

In [16]:
stop_words = spacy.lang.it.stop_words.STOP_WORDS
stop_words_to_remove = ['anni', 'anno']
stop_words = [word for word in stop_words if word not in stop_words_to_remove]


def cleaning(text):
    words_new = [word[1:].translate(str.maketrans('', '', string.punctuation)) 
            if (word.startswith('#') or word.startswith('@')) else word.lower().translate(str.maketrans('', '', string.punctuation))
            for word in text.split()
            if not (re.match(r"http\S+|www\S+|https\S+", word) or word.lower() in stop_words)]
    filtered_text = ' '.join(words_new)
    return filtered_text

In [17]:
tweets[0]

'@StefanoGuerrera Sono una figlia, da piccola non mi hanno insegnato a baciare sulle labbra, mai fatto. Ora ho 30 anni, qualche volta con mia madre lo facevo, per affetto, ed ora che non c’è più lo rimpiango, mai vergognata e mai avuto problemi di sessualità, se ci fosse ancora lo farei ...'

In [18]:
print(cleaning(tweets[0]))

StefanoGuerrera figlia piccola insegnato baciare labbra fatto 30 anni madre facevo affetto c’è rimpiango vergognata problemi sessualità 


In [21]:
def lemmatizationSpacy(text):

    tokens = it_nlp_spacy(text)
    lemmatized_tweet = " ".join([token.lemma_ for token in tokens])
    
    return lemmatized_tweet

In [20]:
print(lemmatizationSpacy(cleaning(tweets[0])))

StefanoGuerrera figlio piccolo insegnato baciare labbra fatto 30 anno madre facevo affetto ci essere rimpiango vergognare problema sessualità


In [22]:
def lemmatizationStanza(text):
    doc = it_nlp_stanza(text)
    lemmatized_tweet = ' '.join([word.lemma if word.lemma.endswith((',', '.')) else word.lemma + ' ' for sent in doc.sentences for word in sent.words])
    
    return lemmatized_tweet

In [23]:
print(lemmatizationStanza(cleaning(tweets[0])))

StefanoGuerrera  figlio  piccolo  insegnare  baciare  labbro  fare  30  anno  madre  fare  affetto  ci  essere  rimpiangere  vergognato  problema  sessualità 


In [8]:
def get_tweets_from_db(chunk_size):
    
    # Create a database connection
    conn = sqlite3.connect('/g100_work/IscrC_mental/data/database/MENTALISM.db')
    cursor = conn.cursor()
    
    cursor.execute(f"SELECT tweet_id, text FROM tweets LIMIT {chunk_size}")
    rows = cursor.fetchall()

    # Create a DataFrame from the fetched rows
    chunk_df = pd.DataFrame(rows, columns=['tweet_id', 'text'])
    
    return chunk_df
    

In [12]:
    conn = sqlite3.connect('/g100_work/IscrC_mental/data/database/MENTALISM.db')
    cursor = conn.cursor()

cursor.execute(f"SELECT tweet_id, text, language FROM tweets LIMIT 1000")
rows = cursor.fetchall()

# Create a DataFrame from the fetched rows
chunk_df = pd.DataFrame(rows, columns=['tweet_id', 'text', 'lang'])

In [13]:
chunk_df

,tweet_id,text,lang
0,53366,I'm in NY working on some cool stuff! ;-P,en
1,94933,www,qst
2,614113,Now I'll begin to work.\r\nAnd i've just signed on twitter.\r\nHallo to everybody!\r\n,en
3,661263,"I'll begin to work now, like jesterday.",en
4,666033,I've just had a coffee.,en
...,...,...,...
995,5476148,"News: Microsoft, Novell Announce First Road Map http://tinyurl.com/37xarp",en
996,5476177,News: http://tinyurl.com/2smfv6 Meridiana comunica di avere messo in vendita un tot di biglietti per Pasqua (periodo: 2-11 aprile) a partire,it
997,5476214,"News: http://tinyurl.com/2hsym2 In occasione della festa della donna del 8 marzo Evolavia fa pagare alle donne solo le tasse, mentre l'accom",it
998,5476255,"News: http://tinyurl.com/254k9x Firefox has reached a new milestone: 300,000,000 downloads since its initial release back in November 9, 200",en


In [14]:
chunk_df.to_csv('test_tweets.csv')

In [35]:
k['text'] = k['text'].apply(cleaning)

In [34]:
stop_words = spacy.lang.it.stop_words.STOP_WORDS
stop_words_to_remove = ['anni', 'anno']
stop_words = [word for word in stop_words if word not in stop_words_to_remove]


def cleaning(text):
    words_new = [word[1:].translate(str.maketrans('', '', string.punctuation)) 
            if (word.startswith('#') or word.startswith('@')) else word.lower().translate(str.maketrans('', '', string.punctuation))
            for word in text.split()
            if not (re.match(r"http\S+|www\S+|https\S+", word) or word.lower() in stop_words)]
    filtered_text = ' '.join(words_new)
    return filtered_text


k['text'] = k['text'].apply(cleaning())

TypeError: cleaning() missing 1 required positional argument: 'text'

In [29]:
print(get_tweets_from_db(20))

    tweet_id  \
0      53366   
1      94933   
2     614113   
3     661263   
4     666033   
5     666103   
6     827173   
7     992993   
8    1002833   
9    1120053   
10   1490743   
11   1542453   
12   1544283   
13   1794873   
14   1857393   
15   1875613   
16   1879593   
17   1879973   
18   1892333   
19   1892983   

                                                                                      text  
0                                                I'm in NY working on some cool stuff! ;-P  
1                                                                                      www  
2   Now I'll begin to work.\r\nAnd i've just signed on twitter.\r\nHallo to everybody!\r\n  
3                                                  I'll begin to work now, like jesterday.  
4                                                                  I've just had a coffee.  
5                                           I think in a few minute it will rain here.....  
6           

In [ ]:
def loop_through_db(
        db_file,
        table_name,
        chunk_size,
        max_tweets_in_file=1000000,
        fout=None,
        ):
    
    # Create a database connection
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Get the total number of rows
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    total_rows = cursor.fetchone()[0]

    # Initialize an empty DataFrame
    result_df = pd.DataFrame()

    # Initialize a tqdm progress bar
    progress_bar = tqdm(total=total_rows, unit="row", desc="Processing")

    n_saved_files=0
    # Loop through the data in chunks
    for offset in range(0, total_rows, chunk_size):
        # Query the database for a chunk of rows
        cursor.execute(f"SELECT tweet_id, text FROM {table_name} LIMIT {chunk_size} OFFSET {offset}")
        rows = cursor.fetchall()

        # Create a DataFrame from the fetched rows
        chunk_df = pd.DataFrame(rows, columns=['tweet_id', 'text'])
        
        
        

        
        # Update the progress bar
        progress_bar.update(len(rows))

    # Close the tqdm progress bar
    progress_bar.close()

    # Close the database connection
    conn.close()

if __name__ == '__main__':
    fire.Fire(get_info_by_user_ids)

In [ ]:
# Create a database connection
conn = sqlite3.connect('/g100_work/IscrC_mental/data/database/MENTALISM.db')
cursor = conn.cursor()

cursor.execute(f"SELECT COUNT(*) FROM tweets")
total_rows = cursor.fetchone()[0]

# Initialize an empty DataFrame
result_df = pd.DataFrame()

# Initialize a tqdm progress bar
progress_bar = tqdm(total=total_rows, unit="row", desc="Processing")

n_saved_files=0


    # Loop through the data in chunks
    for offset in range(0, total_rows, chunk_size):
        # Query the database for a chunk of rows
        cursor.execute(f"SELECT * FROM {table_name} LIMIT {chunk_size} OFFSET {offset}")
        rows = cursor.fetchall()

        # Create a DataFrame from the fetched rows
        chunk_df = pd.DataFrame(rows, columns=column_names)
        
        # Remove the unwanted columns
        if remove_columns is not None:
            chunk_df = chunk_df.drop(columns=remove_columns)

        # Append to the result DataFrame
        # chunk_df['user_id'] = chunk_df['user_id'].astype(str)
        result_df = pd.concat([result_df, chunk_df[chunk_df['user_id'].isin(user_ids_to_retrieve)]], ignore_index=True)

        if len(result_df) >= max_tweets_in_file:
            # Save results to pickle
            print(f'Saving {len(result_df)} tweets to file {n_saved_files}')
            result_df.to_pickle(f'{fout}_{n_saved_files}.pkl')
            # Reset the result DataFrame
            result_df = pd.DataFrame()
            n_saved_files += 1
        
        # Update the progress bar
        progress_bar.update(len(rows))

    # Close the tqdm progress bar
    progress_bar.close()

    # Close the database connection
    conn.close()

if __name__ == '__main__':
    fire.Fire(get_info_by_user_ids)



In [ ]:
it_nlp_spacy = spacy.load('it_core_news_lg')

components_to_remove = ['tagger', 'parser', 'attribute_ruler', 'ner']
for component in components_to_remove:
    it_nlp_spacy.remove_pipe(component)
    
it_nlp_stanza = stanza.Pipeline('it', processors='tokenize,mwt,pos,lemma', verbose=False, use_gpu=False)

stop_words = spacy.lang.it.stop_words.STOP_WORDS
stop_words_to_remove = ['anni', 'anno']
stop_words = [word for word in stop_words if word not in stop_words_to_remove]


def cleaning(text):
    words_new = [word[1:].translate(str.maketrans('', '', string.punctuation)) 
            if (word.startswith('#') or word.startswith('@')) else word.lower().translate(str.maketrans('', '', string.punctuation))
            for word in text.split()
            if not (re.match(r"http\S+|www\S+|https\S+", word) or word.lower() in stop_words)]
    filtered_text = ' '.join(words_new)
    return filtered_text

def lemmatizationSpacy(text):
    tokens = it_nlp_spacy(text)
    lemmatized_tweet = " ".join([token.lemma_ for token in tokens])
    return lemmatized_tweet

def lemmatizationStanza(text):
    doc = it_nlp_stanza(text)
    lemmatized_tweet = ' '.join([word.lemma if word.lemma.endswith((',', '.')) else word.lemma + ' ' for sent in doc.sentences for word in sent.words])
    return lemmatized_tweet


def main():
    parser = argparse.ArgumentParser(description="Customizable pipeline")
    parser.add_argument('--cleaning', action='store_true', help='Run cleaning')
    parser.add_argument('--lemmatizationSpacy', action='store_true', help='Run lemmatization with SpaCy')
    parser.add_argument('--lemmatizationStanza', action='store_true', help='Run lemmatization with Stanza')

    args = parser.parse_args()

    if args.cleaning:
        cleaning()

    if args.lemmatizationSpacy:
        lemmatizationSpacy()

    if args.lemmatizationStanza:
        lemmatizationStanza()

if __name__ == "__main__":
    main()


In [97]:
nlp = spacy.load('it_core_news_lg')

components_to_remove = ['tagger', 'parser', 'attribute_ruler', 'ner']
for component in components_to_remove:
    nlp.remove_pipe(component)

from spacy.language import Language

@Language.component("cleaning")
def cleaning(doc):
    stop_words = spacy.lang.it.stop_words.STOP_WORDS
    new_tokens = [token.text for token in doc if not 
                  (re.match(r"http\S+|www\S+|https\S+|@\S+|#(?!\w)", token.text) or token.is_punct or token.text.lower() in stop_words)]
    return spacy.tokens.Doc(doc.vocab, words=new_tokens)


nlp.add_pipe("cleaning", name="cleaning", first=True)
print(nlp.pipe_names)



['cleaning', 'tok2vec', 'morphologizer', 'lemmatizer']


In [98]:
lemmatized_tweets_spacy = []
for tweet in tweets:
    tokens = nlp(tweet)
    lemmatized_tweet = " ".join([token.lemma_ for token in tokens])
    lemmatized_tweets_spacy.append(lemmatized_tweet)

In [100]:
df_to_check = pd.DataFrame({'original': tweets, 'lemmatized_spacy':lemmatized_tweets_spacy })

In [101]:
df_to_check

,original,lemmatized_spacy
0,"@StefanoGuerrera Sono una figlia, da piccola non mi hanno insegnato a baciare sulle labbra, mai fatto. Ora ho 30 anni, qualche volta con mia madre lo facevo, per affetto, ed ora che non c’è più lo rimpiango, mai vergognata e mai avuto problemi di sessualità, se ci fosse ancora lo farei ...",figlio piccolo insegnato baciare labbro 30 madre affetto ci rimpiango vergognare problema sessualità
1,"Per tutti i followers...... non sono il front man dei Dari, sono un informatico ed ho 42 anni, il Dario che cercate é un mio omonimo.",il followers front man Dari informatico 42 Dario cercare é omonimo
2,@ladyonorato Ho 51 anni e non mi è mai capitato di provare così tanto odio verso il governo!,51 capitare provare odio
3,@carmelitadurso ciao barbara ho ho seguito l'intervento di lemme io sono grassa ho 53 anni sono pronta a mettermi in gioco se mi seguì tu,ciao Barbara intervento lemme grasso 53 pronto mettere mi gioco seguire
4,"@PietroF70 Diplomato alla scuola alberghiera,ho fatto aiuto cuoco, lavapiatti,portiere notturno e istruttore di scuola guida e perché ho 42 anni a casa",diplomato scuola alberghiero aiuto cuoco Lavapiatti portiere notturno istruttore scuola guidare 42
...,...,...
1114,"Mi chiamo Marianna, ho 28 anni e vorrei credere nel mio voto. Ma ahimè, tristemente andrò a votare quello che fa meno schifo, forse.",chiare Marianna 28 volere credere voto tristemente andrò votare schifo
1115,"@civati Populismo di sinistra, populismo di destraAggrappati all’unica vera figura seria, l’attuale presidente del consiglio.Non credo di aver mai letto/sentito da quando sono nato (e ho 47 anni):”facciamo come l’Italia” detto dai tedeschi. Ci vuole preparazione/cultura per fare politica",populismo sinistra populismo destraAggrappare a il unico vero figura serio il attuale presidente credere letto sentire nascere 47 anni):”faccare il Italia tedesco volere preparazione cultura politico
1116,nan,nan
1117,@rosita17rosita ho 49anni e vorrei diventare un politico senza compenso solo per risolvere i problemi dell'Italia,49anni volere politico compenso risolvere il problema Italia


STANZAAA

In [102]:
tweets[0]

'@StefanoGuerrera Sono una figlia, da piccola non mi hanno insegnato a baciare sulle labbra, mai fatto. Ora ho 30 anni, qualche volta con mia madre lo facevo, per affetto, ed ora che non c’è più lo rimpiango, mai vergognata e mai avuto problemi di sessualità, se ci fosse ancora lo farei ...'

In [6]:
it_nlp = stanza.Pipeline('it', processors='tokenize,mwt,pos,lemma', verbose=False, use_gpu=False)

In [103]:
#### just a test code for new tokinizer

stop_words = spacy.lang.it.stop_words.STOP_WORDS
stop_words_to_remove = ['anni', 'anno']
stop_words = [word for word in stop_words if word not in stop_words_to_remove]

sentence = '@StefanoGuerrera Sono una figlia, da piccola non mi hanno insegnato a baciare sulle labbra, mai fatto. Ora ho 30 anni, qualche volta con mia madre lo facevo, per affetto, ed ora che non c’è più lo rimpiango, mai vergognata e mai avuto problemi di sessualità, se ci fosse ancora lo farei ...'

pipeline = stanza.Pipeline('it', processors='tokenize')
doc = pipeline(sentence)


for sentence in doc.sentences:
    sentence.words = [
        word.text[1:] if word.text.startswith('#') else word
        for word in sentence.words if not (
            ### different for @ and # since stanza detaches @ as a sepaate token but not #
            word.text.startswith('@') or
            re.match(r"http\S+|www\S+|https\S+", word.text) or
            word.text.lower() in stop_words
        )
    ]

for sentence in doc.sentences:
    for tok in sentence.words:
        print(tok)

{
  "id": 2,
  "text": "StefanoGuerrera",
  "start_char": 1,
  "end_char": 16
}
{
  "id": 5,
  "text": "figlia",
  "start_char": 26,
  "end_char": 32
}
{
  "id": 6,
  "text": ",",
  "start_char": 32,
  "end_char": 33
}
{
  "id": 8,
  "text": "piccola",
  "start_char": 37,
  "end_char": 44
}
{
  "id": 12,
  "text": "insegnato",
  "start_char": 58,
  "end_char": 67
}
{
  "id": 14,
  "text": "baciare",
  "start_char": 70,
  "end_char": 77
}
{
  "id": 17,
  "text": "labbra",
  "start_char": 84,
  "end_char": 90
}
{
  "id": 18,
  "text": ",",
  "start_char": 90,
  "end_char": 91
}
{
  "id": 21,
  "text": ".",
  "start_char": 101,
  "end_char": 102
}
{
  "id": 3,
  "text": "30",
  "start_char": 110,
  "end_char": 112
}
{
  "id": 4,
  "text": "anni",
  "start_char": 113,
  "end_char": 117
}
{
  "id": 5,
  "text": ",",
  "start_char": 117,
  "end_char": 118
}
{
  "id": 10,
  "text": "madre",
  "start_char": 141,
  "end_char": 146
}
{
  "id": 13,
  "text": ",",
  "start_char": 156,
  "end_char"

In [111]:
### working version of cleaning 

import string
text = tweets[0]
words_new = [word[1:].translate(str.maketrans('', '', string.punctuation)) 
            if (word.startswith('#') or word.startswith('@')) else word.lower().translate(str.maketrans('', '', string.punctuation))
            for word in text.split()
            if not (re.match(r"http\S+|www\S+|https\S+", word) or word.lower() in stop_words)]
filtered_text = ' '.join(words_new)
filtered_text

'StefanoGuerrera figlia piccola insegnato baciare labbra fatto 30 anni madre facevo affetto c’è rimpiango vergognata problemi sessualità '

In [110]:
text

'@StefanoGuerrera Sono una figlia, da piccola non mi hanno insegnato a baciare sulle labbra, mai fatto. Ora ho 30 anni, qualche volta con mia madre lo facevo, per affetto, ed ora che non c’è più lo rimpiango, mai vergognata e mai avuto problemi di sessualità, se ci fosse ancora lo farei ...'

In [113]:
pipeline = stanza.Pipeline('it', processors='tokenize')
doc = pipeline(filtered_text)
        
for i, sentence in enumerate(doc.sentences):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

====== Sentence 1 tokens =======
id: (1,)	text: StefanoGuerrera
id: (2,)	text: figlia
id: (3,)	text: piccola
id: (4,)	text: insegnato
id: (5,)	text: baciare
id: (6,)	text: labbra
id: (7,)	text: fatto
id: (8,)	text: 30
id: (9,)	text: anni
id: (10,)	text: madre
id: (11,)	text: facevo
id: (12,)	text: affetto
id: (13,)	text: c’
id: (14,)	text: è
id: (15,)	text: rimpiango
id: (16,)	text: vergognata
id: (17,)	text: problemi
id: (18,)	text: sessualità


In [17]:
"""
Processor for performing tokenization
"""

import io
import logging

import torch

from stanza.models.tokenization.data import TokenizationDataset
from stanza.models.tokenization.trainer import Trainer
from stanza.models.tokenization.utils import output_predictions
from stanza.pipeline._constants import *
from stanza.pipeline.processor import UDProcessor, register_processor
from stanza.pipeline.registry import PROCESSOR_VARIANTS
from stanza.models.common import doc

# these imports trigger the "register_variant" decorations
from stanza.pipeline.external.jieba import JiebaTokenizer
from stanza.pipeline.external.spacy import SpacyTokenizer
from stanza.pipeline.external.sudachipy import SudachiPyTokenizer
from stanza.pipeline.external.pythainlp import PyThaiNLPTokenizer

logger = logging.getLogger('stanza')

# class for running the tokenizer
@register_processor(name= TOKENIZE)
class TokenizeProcessor(UDProcessor):

    # set of processor requirements this processor fulfills
    PROVIDES_DEFAULT = set([TOKENIZE])
    # set of processor requirements for this processor
    REQUIRES_DEFAULT = set([])
    # default max sequence length
    MAX_SEQ_LENGTH_DEFAULT = 1000

    def _set_up_model(self, config, pipeline, device):
        # set up trainer
        if config.get('pretokenized'):
            self._trainer = None
        else:
            self._trainer = Trainer(model_file=config['model_path'], device=device)

        # get and typecheck the postprocessor
        postprocessor = config.get('postprocessor')
        if postprocessor and callable(postprocessor):
            self._postprocessor = postprocessor
        elif not postprocessor:
            self._postprocessor = None
        else:
            raise ValueError("Tokenizer recieved 'postprocessor' option of unrecognized type; postprocessor must be callable. Got %s" % postprocessor)

    def process_pre_tokenized_text(self, input_src):
        """
        Pretokenized text can be provided in 2 manners:

        1.) str, tokenized by whitespace, sentence split by newline
        2.) list of token lists, each token list represents a sentence

        generate dictionary data structure
        """
        words_new = [word[1:].translate(str.maketrans('', '', string.punctuation)) 
            if (word.startswith('#') or word.startswith('@')) else word.lower().translate(str.maketrans('', '', string.punctuation))
            for word in input_src.split()
            if not (re.match(r"http\S+|www\S+|https\S+", word) or word.lower() in stop_words)]
        input_src = ' '.join(words_new)
        
        document = []
        
        if isinstance(input_src, str):
            sentences = [sent.strip().split() for sent in input_src.strip().split('\n') if len(sent.strip()) > 0]
        elif isinstance(input_src, list):
            sentences = input_src
        idx = 0
        for sentence in sentences:
            sent = []
            for token_id, token in enumerate(sentence):
                sent.append({doc.ID: (token_id + 1, ), doc.TEXT: token, doc.MISC: f'start_char={idx}|end_char={idx + len(token)}'})
                idx += len(token) + 1
            document.append(sent)
        raw_text = ' '.join([' '.join(sentence) for sentence in sentences])
        return raw_text, document

    def process(self, document):
        if not (isinstance(document, str) or isinstance(document, doc.Document) or (self.config.get('pretokenized') or self.config.get('no_ssplit', False))):
            raise ValueError("If neither 'pretokenized' or 'no_ssplit' option is enabled, the input to the TokenizerProcessor must be a string or a Document object.  Got %s" % str(type(document)))

        if isinstance(document, doc.Document):
            if self.config.get('pretokenized'):
                return document
            document = document.text

        if self.config.get('pretokenized'):
            raw_text, document = self.process_pre_tokenized_text(document)
            return doc.Document(document, raw_text)

        if hasattr(self, '_variant'):
            return self._variant.process(document)

        raw_text = '\n\n'.join(document) if isinstance(document, list) else document

        max_seq_len = self.config.get('max_seqlen', TokenizeProcessor.MAX_SEQ_LENGTH_DEFAULT)

        # set up batches
        batches = TokenizationDataset(self.config, input_text=raw_text, vocab=self.vocab, evaluation=True, dictionary=self.trainer.dictionary)
        # get dict data
        with torch.no_grad():
            _, _, _, document = output_predictions(None, self.trainer, batches, self.vocab, None,
                                                   max_seq_len,
                                                   orig_text=raw_text,
                                                   no_ssplit=self.config.get('no_ssplit', False),
                                                   num_workers = self.config.get('num_workers', 0),
                                                   postprocessor = self._postprocessor)

        # replace excessively long tokens with <UNK> to avoid downstream GPU memory issues in POS
        for sentence in document:
            for token in sentence:
                if len(token['text']) > max_seq_len:
                    token['text'] = "<UNK>"

        return doc.Document(document, raw_text)

    def bulk_process(self, docs):
        """
        The tokenizer cannot use UDProcessor's sentence-level cross-document batching interface, and requires special handling.
        Essentially, this method concatenates the text of multiple documents with "\n\n", tokenizes it with the neural tokenizer,
        then splits the result into the original Documents and recovers the original character offsets.
        """
        if hasattr(self, '_variant'):
            return self._variant.bulk_process(docs)

        if self.config.get('pretokenized'):
            res = []
            for document in docs:
                raw_text, document = self.process_pre_tokenized_text(document.text)
                res.append(doc.Document(document, raw_text))
            return res

        combined_text = '\n\n'.join([thisdoc.text for thisdoc in docs])
        processed_combined = self.process(doc.Document([], text=combined_text))

        # postprocess sentences and tokens to reset back pointers and char offsets
        charoffset = 0
        sentst = senten = 0
        for thisdoc in docs:
            while senten < len(processed_combined.sentences) and processed_combined.sentences[senten].tokens[-1].end_char - charoffset <= len(thisdoc.text):
                senten += 1

            sentences = processed_combined.sentences[sentst:senten]
            thisdoc.sentences = sentences
            for sent in sentences:
                # fix doc back pointers for sentences
                sent._doc = thisdoc

                # fix char offsets for tokens and words
                for token in sent.tokens:
                    token._start_char -= charoffset
                    token._end_char -= charoffset
                    if token.words:  # not-yet-processed MWT can leave empty tokens
                        for word in token.words:
                            word._start_char -= charoffset
                            word._end_char -= charoffset

            thisdoc.num_tokens = sum(len(sent.tokens) for sent in sentences)
            thisdoc.num_words = sum(len(sent.words) for sent in sentences)
            sentst = senten

            charoffset += len(thisdoc.text) + 2

        return docs

In [15]:
nlp = stanza.Pipeline(lang='it', processors='TOKENIZE')
doc = nlp(tweets[0])

2023-11-20 15:02:59 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-20 15:02:59 WARNING: Language it package default expects mwt, which has been added
2023-11-20 15:02:59 INFO: Loading these models for language: it (Italian):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |
| tokenize  | combined |
| tokenize  | combined |
| tokenize  | combined |

2023-11-20 15:02:59 INFO: Using device: cpu
2023-11-20 15:02:59 INFO: Loading: tokenize
2023-11-20 15:02:59 INFO: Loading: mwt
2023-11-20 15:02:59 INFO: Loading: tokenize
2023-11-20 15:02:59 INFO: Loading: tokenize
2023-11-20 15:02:59 INFO: Loading: tokenize
2023-11-20 15:02:59 INFO: Done loading processors!


In [16]:
for sentence in doc.sentences:
    for token in sentence.tokens:
        print(token.text)

@
StefanoGuerrera
Sono
una
figlia
,
da
piccola
non
mi
hanno
insegnato
a
baciare
sulle
labbra
,
mai
fatto
.
Ora
ho
30
anni
,
qualche
volta
con
mia
madre
lo
facevo
,
per
affetto
,
ed
ora
che
non
c’
è
più
lo
rimpiango
,
mai
vergognata
e
mai
avuto
problemi
di
sessualità
,
se
ci
fosse
ancora
lo
farei
...


In [10]:
nlp = stanza.Pipeline(lang='it', processors='tokenize')

# Process the text
doc = nlp(tweets[0])

# Access tokenized sentences and tokens
for sentence in doc.sentences:
    for token in sentence.tokens:
        print(token.text)

2023-11-20 14:54:04 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-20 14:54:05 WARNING: Language it package default expects mwt, which has been added
2023-11-20 14:54:05 INFO: Loading these models for language: it (Italian):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |
| tokenize  | combined |
| tokenize  | combined |

2023-11-20 14:54:05 INFO: Using device: cpu
2023-11-20 14:54:05 INFO: Loading: tokenize
2023-11-20 14:54:05 INFO: Loading: mwt
2023-11-20 14:54:05 INFO: Loading: tokenize
2023-11-20 14:54:05 INFO: Loading: tokenize
2023-11-20 14:54:05 INFO: Done loading processors!


@
StefanoGuerrera
Sono
una
figlia
,
da
piccola
non
mi
hanno
insegnato
a
baciare
sulle
labbra
,
mai
fatto
.
Ora
ho
30
anni
,
qualche
volta
con
mia
madre
lo
facevo
,
per
affetto
,
ed
ora
che
non
c’
è
più
lo
rimpiango
,
mai
vergognata
e
mai
avuto
problemi
di
sessualità
,
se
ci
fosse
ancora
lo
farei
...


In [90]:
from stanza.pipeline.processor import UDProcessor, register_processor

In [91]:
@register_processor(name=TOKENIZE)
class TokenizeProcessor(UDProcessor):

 
    PROVIDES_DEFAULT = set([TOKENIZE])
    REQUIRES_DEFAULT = set([])
    
    def process_pre_tokenized_text(self, input_src):
        words_new = [word[1:].translate(str.maketrans('', '', string.punctuation)) 
            if (word.startswith('#') or word.startswith('@')) else word.lower().translate(str.maketrans('', '', string.punctuation))
            for word in input_src.split()
            if not (re.match(r"http\S+|www\S+|https\S+", word) or word.lower() in stop_words)]
        filtered_text = ' '.join(words_new)


KeyboardInterrupt



In [89]:
@register_processor_variant('Tokenizer', 'Cleaning')
class CleaningTokenizer(ProcessorVariant):

    OVERRIDE = True

    def __init__(self, lang):
        pass

    def process(self, text):
        words_new = [word[1:].translate(str.maketrans('', '', string.punctuation)) 
            if (word.startswith('#') or word.startswith('@')) else word.lower().translate(str.maketrans('', '', string.punctuation))
            for word in text.split()
            if not (re.match(r"http\S+|www\S+|https\S+", word) or word.lower() in stop_words)]
        filtered_text = ' '.join(text_new)
        
        doc = self.tokenizer_pipeline(filtered_text)
        
        return document

In [86]:
from stanza.pipeline.processor import ProcessorVariant, register_processor_variant
from stanza.models.common.doc import Document

In [64]:
@register_processor_variant('tokenize', 'custom_clean')
class CustomTokenizer(ProcessorVariant):
    OVERRIDE = True

    def __init__(self, config):
        self.tokenizer_pipeline = stanza.Pipeline('it', processors='tokenize')

    def process(self, text):
        doc = self.tokenizer_pipeline(text)
        for sentence in doc.sentences:
            sentence.words = [
                word for word in sentence.words if not (
                    re.match(r"http\S+|www\S+|https\S+|@\S+|#(?!\w)", word.text) or
                    word.text.lower() in stop_words
                )
            ]

        return doc

# Input sentence
sentence = '@ladyonorato Ho 51 anni e non mi è mai capitato di provare così tanto odio verso il governo!'

# Define stop words
stop_words = spacy.lang.it.stop_words.STOP_WORDS
stop_words_to_remove = ['anni', 'anno']
stop_words = [word for word in stop_words if word not in stop_words_to_remove]

# Create a Stanza pipeline with the custom tokenizer
custom_pipeline = stanza.Pipeline('it', processors={'tokenize': 'custom_clean'})

# Process the sentence using the custom pipeline
custom_doc = custom_pipeline(sentence)

# Access tokenized words in the modified document
modified_tokenized_words = [word.text for sentence in custom_doc.sentences for word in sentence.words]
print(modified_tokenized_words)


['@', 'ladyonorato', 'Ho', '51', 'anni', 'e', 'non', 'mi', 'è', 'mai', 'capitato', 'di', 'provare', 'così', 'tanto', 'odio', 'verso', 'il', 'governo', '!']


In [61]:
for sentence in doc.sentences:
    for word in sentence.words:
        print(word.text, word.lemma)

ladyonorato None
Ho None
51 None
e None
non None
mi None
è None
mai None
capitato None
di None
provare None
così None
odio None
verso None
il None
governo None
! None


In [65]:
import re
import stanza
from stanza.pipeline.processor import ProcessorVariant, register_processor_variant
from stanza.models.common.doc import Document

# Download the Italian language model
stanza.download('it')

@register_processor_variant('tokenize', 'custom_clean')
class CustomTokenizer(ProcessorVariant):
    OVERRIDE = True

    def __init__(self, config):
        # Create a Stanza pipeline for Italian with only the tokenizer
        self.tokenizer_pipeline = stanza.Pipeline('it', processors='tokenize')

    def process(self, text):
        # Process the text using the tokenizer pipeline
        doc = self.tokenizer_pipeline(text)

        # Define stop words
        stop_words = ['anno', 'anni', 'tanto']

        # Delete tokens based on specified conditions
        for sentence in doc.sentences:
            sentence.words = [
                word for word in sentence.words if not (
                    word.text.startswith('@') or
                    word.text.startswith('#') or
                    re.match(r"http\S+|www\S+|https\S+|@\S+|#(?!\w)", word.text) or
                    word.text.lower() in stop_words
                )
            ]

        return doc

# Input sentence
sentence = '@ladyonorato Ho 51 anni e non mi è mai capitato di provare così tanto odio verso il governo!'

# Create a Stanza pipeline with the custom tokenizer
custom_pipeline = stanza.Pipeline('it', processors={'tokenize': 'custom_clean'})

# Process the sentence using the custom pipeline
custom_doc = custom_pipeline(sentence)

# Access tokenized words in the modified document
modified_tokenized_words = [word.text for sentence in custom_doc.sentences for word in sentence.words]
print(modified_tokenized_words)


['@', 'ladyonorato', 'Ho', '51', 'anni', 'e', 'non', 'mi', 'è', 'mai', 'capitato', 'di', 'provare', 'così', 'tanto', 'odio', 'verso', 'il', 'governo', '!']


In [51]:
import stanza

# Download the Italian language model
stanza.download('it')

# Create a Stanza pipeline for Italian
pipeline = stanza.Pipeline('it', processors='tokenize')

# Input sentence
sentence = '@ladyonorato Ho 51 anni e non mi è mai capitato di provare così tanto odio verso il governo!'

# Process the sentence using the pipeline
doc = pipeline(sentence)

# Access tokenized words
tokenized_words = [word.text for sentence in doc.sentences for word in sentence.words]

print(tokenized_words)



['@', 'ladyonorato', 'Ho', '51', 'anni', 'e', 'non', 'mi', 'è', 'mai', 'capitato', 'di', 'provare', 'così', 'tanto', 'odio', 'verso', 'il', 'governo', '!']


In [45]:
custom_pipeline = stanza.Pipeline(lang='it', processors={'tokenize': 'CleaningTokenizing', 'mwt': 'default', 'pos': 'default',
                                                         'lemma': 'default'}, verbose=False, use_gpu=False)

# Example usage
text = '@ladyonorato Ho 51 anni e non mi è mai capitato di provare così tanto odio verso il governo!'
doc = custom_pipeline(text)

# Accessing tokenized words from the custom processor
tokenized_words = [word.text for sentence in doc.sentences for word in sentence.words]
print(tokenized_words)

['@', 'ladyonorato', 'Ho', '51', 'anni', 'e', 'non', 'mi', 'è', 'mai', 'capitato', 'di', 'provare', 'così', 'tanto', 'odio', 'verso', 'il', 'governo', '!']


In [46]:
for sentence in doc.sentences:
    for word in sentence.words:
        print(word.text, word.lemma)


@ @
ladyonorato ladyonorato
Ho avere
51 51
anni anno
e e
non non
mi mi
è essere
mai mai
capitato capitare
di di
provare provare
così così
tanto tanto
odio odiare
verso verso
il il
governo governo
! !


In [ ]:
it_nlp = stanza.Pipeline('it', processors='tokenize,mwt,pos,lemma', verbose=False, use_gpu=False)

In [ ]:
it_nlp = stanza.Pipeline('it', processors={'tokenize': MyCustomTokenizer, 'mwt': 'default', 'pos': 'default', 'lemma': 'default'}, verbose=False, use_gpu=False)

In [ ]:
### pipeline with tdqm because it takes a lot of time to process 

import time
from tqdm.notebook import tqdm

lemmatized_tweets_stanza = []
total_start_time = time.time() 

progress_bar = tqdm(total=len(tweets), desc="Processing tweets", unit="tweet", leave=False)

for tweet in tweets_no_hashtags:
    doc = it_nlp(tweet)
    lemmatized_tweet = ' '.join([word.lemma if word.lemma.endswith((',', '.')) else word.lemma + ' ' for sent in doc.sentences for word in sent.words])
    lemmatized_tweets_stanza.append(lemmatized_tweet)
    

    progress_bar.update(1)


progress_bar.close()

total_end_time = time.time()
total_time = total_end_time - total_start_time
average_time = total_time / len(tweets) if len(tweets) > 0 else 0

print(f"\nTotal time for all iterations: {total_time:.2f} seconds")
print(f"Average time per item: {average_time:.4f} seconds")
